In [1]:
import re
import spacy
import os
import shutil
from pathlib import Path
nlp = spacy.load("pl_core_news_lg")

In [2]:
"""
Ważna uwaga:
Pliki z lekturami (trzeba funkcji ustawić argument is_fragment=False) do wczytania muszą mieć następującą strukturę:
Autor

Tytuł(dowolna ilość linii)

Ewentualny numer ISBN

Tekst(dowolna ilość linii)
Ewentualna stopka po treście(przypis od wolnychlektur): -----
"""
"""
Przy podawaniu fragmentu(is_fragment=True) skanowany jest cały plik, więc struktura tekstu jest dowolna-nie pobiera wtedy nformacji o autorze
"""
"""
Tworzy worek słów z pliku
file_name - nazwa pliku z której utworzyc worek słów;
is_fragment - True/False - mówi czy podawany plik to fragment, czy cały utwór
with_stop_words - True/False mówiący czy usuwać stop wordsy czy nie
lemma - określa czy worke ma byćzlematyzowany
only_stop_words - mówi czy worek słów ma zawierać tylko stop wordsy(jeśli jest True, to with_stop_words jest bez znaczenia)
"""
def create_bag_of_words_from_file(file_name, is_fragment, with_stop_words=False, lemma=True, only_stop_words=False):
#struktura worka: kluczem jest słowo, a wartością jest liczebność
    bag = {}
    with open(file_name,'r', encoding='utf-8') as file:
#przypisanie autora, jeśli jest to cały utwór- znajduje się pod kluczem '0'
        if not is_fragment:
            bag[0] = get_author(file)
            
        for line in file:
            for word in line.split():
#sprawdzanie czy linia nie jest stopką. Jak jest-skończ działanie
                if not is_fragment and word == "-----":
                    return bag
#jeśli nie jest stopką, to wywołuja funkcję dołączającą linię do worka słów 
            bag = create_bag_of_words_from_string(line, bag, with_stop_words, lemma, only_stop_words)
    return bag

"""
Tworzy worek słów ze stringa- używane do pojedyńczych linii przy wczytywaniu z pliku, lub może być wykorzystane
do tworzenia worka z bezpośrednio podancyh fragmentów
string - tekst, z którego tworzy się worek słów
bag - worek słów, do którego chemy dodać nowe wystąpienia
with_stop_words - czy uwzględniamy stop wordsy
lemma - czy lematyzujemy
only_stop_words - czy worek słów ma sięskładać tylko ze stop wordsów(gdy True, to with_stop_words jest bez znaczenia)
"""
def create_bag_of_words_from_string(string, bag, with_stop_words=False, lemma=True, only_stop_words=False):
    stop_words = []
#tablica znaków interpunkcyjnych do pozbycia się z wyrazów
    punc = '''…!”„()-[]»{};:'"\,<>./?@#$%^&*_~«—'''+ "\n" + " "
#ewentualne przypisanie stopwordsów
    if not with_stop_words or only_stop_words:
        stop_words = get_stop_words_from_file('stopWords.txt')
#wyciągnięcie słów
    words = words_from_line(string, lemma, punc)
    for word in words:
#zamienienie wielkich liter na mniejsze
        word = word.lower()
#jeśli jest stop wordsem- nie uwzględniamy (jeśli with_stop_words było True i only_stop_words było False, 
#to 'stop_words' jest puste, czyli nie wejdzie do ifa)    
        if word in stop_words:
            if only_stop_words:
                bag = add_to_bag(bag, word)
            continue
#jeśli nie uwzględniamy tylko stop wordsów, dodajemy wyraz normalnie do worka
        elif not only_stop_words:
            bag = add_to_bag(bag, word)
    return bag

"""
Funkcja pomocnicza dodająca do worka słów nowe wystąpienie słowa
bag - worek słów, do którego dodajemy
word - słowo, które dodajemy
"""
def add_to_bag(bag, word):
    if (word not in bag) and (len(word) != 0):
        bag[word] = 1
#inkrementacja liczności słowa, jeśli już istnieje w worku
    elif len(word) != 0:
        bag[word] += 1
    return bag

"""
Tworzy tablicę słów w zależności od tego, czy używamy lematyzacji
- Jeżeli nie używamy lematyzacji to usuwa ze słów ewentualne znaki interpunkcyjne
- Jeżeli używamy lematyzacji to usuwa nadmierne spacje, generuje tokeny i zamienia je na słowa
w wersji podstawowej zależnie od kontekstu w linijce. Pomija słowa będące znakami interpunkcyjnymi
"""
def words_from_line(line, lemma, punc):
    words = []
    if not lemma:
        for word in line.split():
            for ele in word:
                if ele in punc:
                    word = word.replace(ele, "")
            words.append(word)
    
    else:
        line = re.sub(' +', ' ', line)
        tokens = nlp(line, disable=["parser", "ner"])
        for token in tokens:
            word = token.lemma_
            if word in punc:
                continue
            words.append(word)
        
    return words

"""
Funkcja łączy worki słów danych autorów, przez co wynikowo powstaje jeden worek dla każdego autora
word_bags - lista stworzonych z lektur worków słów
WAŻNE: w każdym worku pod indeksem 0(liczba, nie string) musi znajdować się autor danego utworu
WAŻNE2: używać przed użyciem funkcji count_percents na workach
struktura outputu: słownik, gdzie kluczem są autorzy, a wartościami worek słów(z których zostanie wyrzucony klucz 0 -z autorem)
"""
def combine_word_bags(word_bags):
    combined = {}
#przechodzenie po wszystkich workach z listy
    for bag in word_bags:
        keys = combined.keys()
#pobieranie autora z aktualnego worka i usuwanie go ze słownika
        author = bag[0]
        del bag[0]
#jeśli autor już jest w wynikowym worku, to aktualizujemy wystąpienia słów u niego
        if author in keys:
            for elem in bag:
#jeśli autor ma już dane słowo, to dodajemy ilość wystąpień z aktualnie iterowanego worka
                if elem in combined[author]:
                    combined[author][elem] += bag[elem]
#jeśli słowa jeszcze nie ma, to dodajemy rekord
                else:
                    combined[author][elem] = bag[elem]
#jeśli autora nie ma jeszcze w wynikowym worku, to dodajemy rekord, przypisując do niego aktualny worek
        else:
            combined[author] = bag
    return combined

"""
Funkcja zmienia ilosć wystąpień danego słowa na procent wartość wystąpień w tekście
word_bag - worek słów do obliczenia procentowych wystąpień
"""
def count_percents(word_bag):
    amount = 0
#zliczanie ilości słów w worku
    for elem in word_bag:
        if elem == 0:
            continue
        amount += word_bag[elem]
#przeliczanie wystąpień na wartości procentowe
    for elem in word_bag:
        if elem == 0:
            continue
        word_bag[elem] = word_bag[elem]/amount*100
    return word_bag

"""
Funkcja zapisuje worki słów do plików. Folderem nadrzędnym jest Bags/, pliki tekstowe mają tytuł autor_worka.txt.
Format pojedyńczej linii z rekordem: słowo ilość_wystąpień
word_bags- słownik worków, gdzie kluczem jest autor, a wartością worek
directory - ścieżka do folderu, gdzie mają być zapisywane worki słów danych autorów 
"""
def save_word_bags_to_file(word_bags, directory):
    if not os.path.isdir('Bags'):
        os.mkdir('Bags')
    os.mkdir(directory)
    for key in word_bags.keys():
        bag = word_bags[key]
        file = open(directory + key + ".txt", "w", encoding="utf-8")
        for b_keys in bag.keys():
            file.write(b_keys + " " + str(bag[b_keys]) + '\n')
        file.close()
      
"""
Funkcja pobiera już wcześniej utworzone worki słów z folderu
directory - folder z workami słów 
WAŻNE:
directory podawać bez ukośnika! Czysta nazwa!
pliki z workami słów danych autorów muszą mieć za nazwę imię i nazwisko tego autora
"""
def get_word_bags_from_directory(directory):
    files = os.listdir(directory)
    bags = {}
    for author in files:
        tmp_bag = {}
        file = open(directory + "\\" + author, encoding='utf-8')
        for line in file:
            line = line.split()
            tmp_bag[line[0]] = int(line[1])
        bags[repr(author).replace(".txt", "").replace("'", "")] = tmp_bag
    return bags
    
            

In [3]:
"""
funkcja pobiera stop wordsy z pliku i zwraca je jako listę
file_name - ścieżka do pliku, gdzie są zapisane stop wordsy
"""
def get_stop_words_from_file(file_name):
    stops = []
    borders = ["\\n", "'"]
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            single_line = repr(line)
            for char in borders:
                single_line = single_line.replace(char,"")
            if not(single_line == ""):
                stops.append(single_line)
    return stops

"""
Funkcja pobiera autora utworu z pierwszej linii, przeskakuje tytuł i przeskakuje number ISBN
file - plik utworu
"""
def get_author(file):
    author = file.readline()
    author = author.replace("\n","")
    author = list(author)
    if author[0] == " ":
        author[0] = ""
    author = "".join(author)
    
    file.readline()
    while not (file.readline()  == "\n"):
        continue
    position_before = file.tell()
    if not file.readline().startswith('ISBN'):
        file.seek(position_before)
    return author

In [4]:
"""
Wywołuje się ją po to, aby uzyskać worki słów dla autorów- uprzednio zapisanych lub stworzone na nowo
Od wartości argumentów zależy nazwa folderu gdzie worki będąszukane/zapisywane
only_stop_words - worki tylko ze stop wordsami(jeśli jest True, to with_stop_words nie ma znaczenia)
with_stop_words - worek słów z uwzględnieniem stop wordsów
lemma - czy lematyzować
"""
def create_bags_of_words_from_authors(only_stop_words, with_stop_words, lemma):
    bags = {}
#tworzenie ścieżki folderu na podstawie argumentów
    path = "Bags/BagOfWords_"
    if only_stop_words:
        path += "only"
    elif with_stop_words:
        path += "with"
    elif not with_stop_words:
        path += "no"
    path += "StopWords_"
    if not lemma:
        path += "Not"
    path += "Lemma/"
#Jeśli taki folder już istnieje, oznacza, że już wcześniej go utworzyliśmy i tylko wczytujemy gotowe worki słów
    if os.path.isdir(path):
        bags = get_word_bags_from_directory(path)
#Jeśli nie ma takiego folderu, to trzeba utworzyć worki
    else:
#pobiera nazwy plików, a więc wszystkie książki
        books_list = os.listdir('Books')
        word_bags = []
#tworzymy worek słów dla każdej książki i dodajemy do listy
        for book in books_list:
            print("doing " + book)
            word_bags.append(create_bag_of_words_from_file('Books/'+book, False, with_stop_words, lemma, only_stop_words))
#łączymy worki słów dla poszczególnych autorów(jeden autor- jeden worek słów)
        bags = combine_word_bags(word_bags)
#zapisujemy worki słów do pliku o utworzonej ścieżce
        save_word_bags_to_file(bags, path)
    return bags